# ES|QL Augmentation: INTO and PROCESS WITH

This notebook demonstrates elastic-script's augmentation of ES|QL with the `INTO` and `PROCESS WITH` commands.

**Key Points:**
- ES|QL itself is **unchanged** - we augment it with new commands
- `INTO` stores query results in a variable or indexes them
- `PROCESS WITH` invokes a procedure for each row or batch

## 1. INTO Variable

Store ES|QL query results directly into a variable. This is the simplest form of ES|QL augmentation.

In [ ]:
-- Store query results in a variable
CREATE PROCEDURE demo_into_variable()
BEGIN
    -- This ES|QL query's results go INTO the variable error_logs
    FROM kibana_sample_data_logs | WHERE response.keyword == '503' | LIMIT 5 INTO error_logs;
    
    -- Now we can use the variable
    PRINT 'Found ' || ARRAY_LENGTH(error_logs) || ' matching logs';
    
    -- Iterate over results
    DECLARE log DOCUMENT;
    FOR log IN error_logs LOOP
        PRINT 'Client: ' || log.clientip || ' - URL: ' || log.url;
    END LOOP
END PROCEDURE

In [ ]:
CALL demo_into_variable()

## 2. INTO with Aggregation

Store aggregated results in a variable.

In [ ]:
CREATE PROCEDURE demo_into_aggregation()
BEGIN
    -- Aggregate and store in variable
    FROM kibana_sample_data_logs 
    | STATS request_count = COUNT(*) BY response.keyword 
    | SORT request_count DESC 
    INTO response_stats;
    
    PRINT 'Response code distribution:';
    
    DECLARE stat DOCUMENT;
    FOR stat IN response_stats LOOP
        PRINT 'Response ' || stat['response.keyword'] || ': ' || stat.request_count || ' requests';
    END LOOP
END PROCEDURE

In [ ]:
CALL demo_into_aggregation()

## 3. INTO Index

Store query results directly into a destination index. Great for ETL pipelines.

In [ ]:
CREATE PROCEDURE demo_into_index()
BEGIN
    -- Archive error responses to a separate index
    FROM kibana_sample_data_logs 
    | WHERE response.keyword == '503' OR response.keyword == '404'
    | LIMIT 10
    INTO 'archived-error-responses';
    
    PRINT 'Error responses archived to archived-error-responses index';
    
    -- Verify by querying the new index
    DECLARE archived = ESQL_QUERY('FROM archived-error-responses | LIMIT 5');
    PRINT 'Archived ' || ARRAY_LENGTH(archived) || ' documents';
END PROCEDURE

In [ ]:
CALL demo_into_index()

## 4. PROCESS WITH

Invoke a procedure for each row in the query results. This is powerful for row-by-row processing.

In [ ]:
-- First, create a procedure that processes each log entry
CREATE PROCEDURE process_log_entry(log DOCUMENT)
BEGIN
    -- This runs for each row
    PRINT 'Processing: [' || log['response.keyword'] || '] ' || log.url;
END PROCEDURE

In [ ]:
CREATE PROCEDURE demo_process_with()
BEGIN
    PRINT '--- Processing logs ---';
    
    -- Process each log entry with our procedure
    FROM kibana_sample_data_logs | LIMIT 5 PROCESS WITH process_log_entry;
    
    PRINT '--- Done ---';
END PROCEDURE

In [ ]:
CALL demo_process_with()

## 5. Summary

| Command | Description | Example |
|---------|-------------|--------|
| `INTO variable` | Store results in a variable | `FROM logs-* \| INTO my_data;` |
| `INTO 'index'` | Index results into destination | `FROM logs-* \| INTO 'archive';` |
| `PROCESS WITH proc` | Call procedure per row | `FROM logs-* \| PROCESS WITH handler;` |
| `PROCESS WITH proc BATCH n` | Call procedure per batch | `FROM logs-* \| PROCESS WITH handler BATCH 50;` |

**Key Benefits:**
- ES|QL remains unchanged - familiar query syntax
- Seamless integration with elastic-script procedures
- Efficient batch processing for large datasets
- Direct indexing without intermediate steps

In [ ]:
-- Cleanup
DELETE PROCEDURE demo_into_variable;
DELETE PROCEDURE demo_into_aggregation;
DELETE PROCEDURE demo_into_index;
DELETE PROCEDURE process_log_entry;
DELETE PROCEDURE demo_process_with;